<a href="https://colab.research.google.com/github/kkcandra/candra/blob/main/Tugas_AI/Uas_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Menghubungkan Dengan google Dive, Ikutin Prosesnya sampai terhubung

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


Sesuakan Isi dari /content/drive/My Drive/Data_Uang Dengan Tempat Dimana menyimpan file dataset ( Data_Uang )

In [ ]:
base_dir = '/content/drive/My Drive/Data_Uang'
!ls "/content/drive/My Drive/Data_Uang"

bahan  train  validation


Membuat Direktori Traning dan Validasi, dan membuat direktori seratus & limapuluh dalam direktori Traning dan Validasi

In [ ]:
import os
# Menentukan Direktori

# Tentukan nama direktori untuk latihan dan validasi
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Buat direktori latihan dan validasi beserta sub-direktorinya (seratus dan limapuluh)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

train_seratus = os.path.join(train_dir, 'seratus')
train_limapuluh = os.path.join(train_dir, 'limapuluh')
validation_seratus = os.path.join(validation_dir, 'seratus')
validation_limapuluh = os.path.join(validation_dir, 'limapuluh')

os.makedirs(train_seratus, exist_ok=True)
os.makedirs(train_limapuluh, exist_ok=True)
os.makedirs(validation_seratus, exist_ok=True)
os.makedirs(validation_limapuluh, exist_ok=True)

# Cek apakah direktori telah berhasil dibuat
print("Direktori latihan dan validasi telah dibuat:")
print("Train Directory:", train_dir)
print("Validation Directory:", validation_dir)

Menyiapkan Dataset, Mengisi Folder Traning dan Validasi dengan data yang ada di dalam Folder Bahan dan melakukan Shuffling data dengan rasio pembagi 90 % data Traning 10 % data Validasi

In [ ]:
import os
import random
from shutil import copyfile
import shutil

# Mendefinisikan path ke folder 'Data_Uang'
data_uang_dir = base_dir   # Ganti dengan path yang sesuai

# Memeriksa apakah folder 'Data_Uang' ada
if not os.path.exists(data_uang_dir):
    print("Folder 'Data_Uang' tidak ditemukan.")
    exit()

# Mendefinisikan path untuk folder 'bahan'
bahan_dir = os.path.join(data_uang_dir, 'bahan')

# Memeriksa apakah folder 'bahan' ada di dalam folder 'Data_Uang'
if not os.path.exists(bahan_dir):
    print("Folder 'bahan' tidak ditemukan di dalam folder 'Data_Uang'.")
    exit()

# Mendefinisikan path untuk folder 'seratus' dan 'limapuluh' di dalam folder 'bahan'
seratus_dir = os.path.join(bahan_dir, 'seratus')
limapuluh_dir = os.path.join(bahan_dir, 'limapuluh')

# Memeriksa apakah folder 'seratus' dan 'limapuluh' ada di dalam folder 'bahan'
if not os.path.exists(seratus_dir) or not os.path.exists(limapuluh_dir):
    print("Folder 'seratus' atau 'limapuluh' tidak ditemukan di dalam folder 'bahan'.")
    exit()

def train_val_split(source, train, val, train_ratio):
    # Membuat direktori jika tidak ada
    if not os.path.exists(train):
        os.makedirs(train)
    if not os.path.exists(val):
        os.makedirs(val)

    total_size = len(os.listdir(source))
    train_size = int(train_ratio * total_size)
    val_size = total_size - train_size

    randomized = random.sample(os.listdir(source), total_size)
    train_files = randomized[0:train_size]
    val_files = randomized[train_size:total_size]

    for i in train_files:
        i_file = os.path.join(source, i)
        destination = os.path.join(train, i)
        copyfile(i_file, destination)

    for i in val_files:
        i_file = os.path.join(source, i)
        destination = os.path.join(val, i)
        copyfile(i_file, destination)

# Pembagian Traning dan Validasi
# Traning 100k
source_00 = seratus_dir
train_00 = train_seratus
val_00 = validation_seratus
train_ratio = 0.9  # Misalkan 90% data untuk training
train_val_split(source_00, train_00, val_00, train_ratio)

# Traning 50K
source_01 = limapuluh_dir
train_01 = train_limapuluh
val_01 = validation_limapuluh
train_val_split(source_01, train_01, val_01, train_ratio)


Mengecek Kembali Apakah jumlah data sudah terisi dengan benar

In [ ]:
print('Jumlah All seratus :', len(os.listdir(source_00)))
print('Jumlah Train seratus :', len(os.listdir(train_seratus)))
print('Jumlah Val seratus :', len(os.listdir(validation_seratus)))

Jumlah All seratus : 100
Jumlah Train seratus : 90
Jumlah Val seratus : 10


Tahap Pre Processing Menggunakan Tensorflo dengan optimizer Adam dan image data generator, digunakan untuk melatih kondisi data
dimana membuat data dengan rescale 1./255 untuk mengenali picxel terkecil, rotasi ketika gambar dalam bentuk miring, filip ketika gambar terbalik dan seterusnya

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
                rescale = 1./255,
                rotation_range = 30,
                horizontal_flip = True,
                shear_range = 0.3,
                fill_mode = 'nearest',
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                zoom_range = 0.1
)

val_datagen = ImageDataGenerator(
                rescale = 1./255,
                rotation_range = 30,
                horizontal_flip = True,
                shear_range = 0.3,
                fill_mode = 'nearest',
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                zoom_range = 0.1
)

Menentukan Targent Dimana Kami menggunakan target size 150, 150 dengan batch size 10, hasil terbagi menjadi 2 kelas yang di atas adalah data dari traning dan yang di bawah untuk hasil data validasi

In [ ]:
train_generator = train_datagen.flow_from_directory(
      train_dir,
      target_size = (150, 150),
      batch_size = 10,
      class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
      validation_dir,
      target_size = (150, 150),
      batch_size = 10,
      class_mode = 'categorical'
)

Callback dengan batasan 99% ketika data sudah diperoses dan menghasilkan akurasi 99%, meskipun iterasi masih berjalan maka pemprosesan akan berhenti

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') > 0.99):
      print('\n Akurasi Mencapai 99%')
      self.model.stop_traning = True

checkpoint = myCallback()

Model CNN

In [ ]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(200, activation = 'relu'),
            tf.keras.layers.Dropout(0.3,seed=112),
            tf.keras.layers.Dense(500, activation = 'relu'),
            tf.keras.layers.Dropout(0.5,seed=112),
            tf.keras.layers.Dense(2, activation = 'sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
            train_generator,
            steps_per_epoch = 6,
            epochs = 25,
            validation_data = val_generator,
            validation_steps = 1,
            verbose = 1,
            callbacks=[checkpoint]
            )



Tingkat Akurasi Dan Loos Model

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_los = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Traning Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accoracy')
plt.title('Traning And Valifation Accuracy')
plt.legend(loc = 'best')
plt.show()

Coba Kalsifikasi

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files
import matplotlib.pyplot as plt  # Anda harus mengimpor pyplot

uploaded = files.upload()

for fn in uploaded.keys():
    path = fn

    # Prediksi Gambar
    img = image.load_img(path, target_size=(150, 150))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=100)

    print(fn)

    class_list = os.listdir(train_dir)

    for j in range(42):
        if classes[0][j] == 1.:
            print(' Gambar Ini ', class_list[j-1])
            break
